In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import serial #Librería para la comunicación serial RS232 (Este es el protocolo de comunicación utilizado por las tarjetas Arduino)

arduino = serial.Serial(None, baudrate=115200, timeout=1) #Creamos un puerto llamado arduino (puede ser otro nombre) con el mismo baudrate (bits por segundo) fijado en el programa del Arduino.

if arduino.isOpen(): #Si el puerto ya estaba abierto, lo cerramos
    print("Se ha cerrado una sesión previa del puerto")
    arduino.close()
arduino.port='COM3' #Este es el puerto que le asignó el sistema operativo a la tarjeta Arduino (Se consulta en la interfaz de Arduino o en el Administrador de Dispositivos (en el caso de Windows))
arduino.open() #Se abre el puerto. Desde este momento ya se pueden enviar y recibir mensajes a la tarjeta Arduino

In [11]:
#Configurar tiempo de integración

int_time = 1000#[ms]

arduino.write('I\n'.encode())
arduino.write(f'{int_time}\n'.encode())

5

In [14]:
%matplotlib qt
#En jupyter se usa esto para crear una gráfica interactiva en lugar de mostrarla debajo de la celda.

N=288 #Número de muestras a almacenar para mostrar en la gráfica
intensity_array=np.zeros(N)
wavelenght_array=np.arange(N)*2.3548+321.47 #Calibración por defecto, puede cambiarse luego
data = []
try:
    while True:
        arduino.reset_input_buffer() #Limpia el búfer
        read_value=arduino.readline()
        intensity_values=read_value.decode() #String con todas las intensidades de cada longitud de onda separadas por coma
        intensity_array=np.fromstring(intensity_values, dtype=int, sep=",")
        prom="promedio= "+str(np.mean(intensity_array).round(2))

        if(len(intensity_array)==289):
            plt.plot(wavelenght_array,intensity_array[:288],'-')
            plt.text(860,155,prom)
            plt.xlabel('Wavelength (nm)')
            plt.ylabel('Intensity (a.u.)')
            plt.grid()
            plt.draw() #Dibuja los puntos de la gráfica
            data.append(intensity_array)
            plt.pause(1) #Los puntos quedan visibles durante este tiempo
            plt.cla() #limpia la gráfica
except KeyboardInterrupt:
    print("Gráfica interrumpida por el usuario")

C:\Users\User\AppData\Local\Temp/ipykernel_17544/4113855156.py:13: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  intensity_array=np.fromstring(intensity_values, dtype=int, sep=",")
c:\Users\User\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\User\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\AppData\Local\Temp/ipykernel_17544/4113855156.py:13: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  intensity_array=np.fromstring(intensity_values, dtype=int, sep=",")
c:\Users\User\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(

Gráfica interrumpida por el usuario


In [15]:
now = datetime.now()
date = now.strftime("%d_%m_%Y__%H_%M_%S")
date

archive = open(f"esp_data\esp_X{len(data)}__{date}_muua2.txt", "w")

data = np.array(data)

for i in data:
    line = " ".join(str(elemento) for elemento in i)
    archive.write(line + "\n")

archive.close()

In [ ]:
arduino.close()